# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 
The CNN-RNN is an encoder-decoder model. The encoder outputs a fixed-size vector, embedding the relevant context of the image.

Encoder: 
I used the default encoder setup. This is based on the resnet50 encoder, but with the last layer replaced by a custom fully connected layer that outputs a vector of length embed_size. Aside from the custom output layer, the entire model is frozen. 
    - Resnet50 made sense because it's a deep network, with a reasonably large image input size, pretrained with many output classes and proven on the COCO dataset. Since we have a limited amount of GPU time, I needed a pretrained model, and this one seemed as good as any. 

The Decoder:
        For training, the RNN decoder takes in a caption (sequence of word indexes) and converts them to a sequence of embedding vectors, each with the same length as the image embedding. Then the image embedding and the sequence of word embeddings are concatenated together and fed as input into the LSTM. The LSTM outputs a sequence of outputs of length "hidden_size". Finally, the hidden features are put through a fully connected layer that outputs a vector of dense vector of length vocab_size, which are effectively guesses about the next word.

Decoder:
        0. The decoder takes in a caption, which is just a list of word indexes, and feature, the image embedding.
        1. Remove <end> token from each caption. The last value of each caption is <end>, and it needs to be removed. This is because the captions will be used as inputs, and sending in <end> would be like asking the model what comes after the end. It just wouldn't make any sense, and we wouldn't have a word to act as a label to train it.
        2. Then each word in the the cropped caption gets converted into an embedding of the same length as the image embedding. 
        3. Concatenate the [image embedding:cropped captions] together and feed that input the LSTM. In practice, we need to unsqueeze the features, since it doesn't have a dimension for word-position. And then we concatenate across the word-position dimension (which is dim=1, since batch is the dim=0). Note that the memory state cells (h0, c0) are initialized to 0.
        4. The LSTM outputs a feature vector of hidden_size. This could have gone into another layer of LSTM, but I left num_layers=1. hidden_size and num_layers are our main ways of controlling the size/complexity of this model. 
          
Selecting Hyperparameters:
    - `batch_size` : 64; I usually start with 32, but I bumped it up to 64 to match the "Show, Attend and Tell" paper
    - `vocab_thresh` : 5; to match the "Show and Tell" paper. Any smaller, and I don't think there would be enough samples for the individual words to build a meaningful embedding. 
    - `vocab_from_file`: False the first time, True after that.
    - `embed_size`: 256; I think the "Show and Tell" paper used 512, but I stuck with the default from notebook 1. The main reason to reduce complexity here is to speed up the model, and that seems to be within the reasonable range we've been discussing in class. 
    - `hidden_size`: 512; This matches the "Show and Tell" model and defaults from part 1. Also, since we're not doing much to reduce overfitting, I'm erring on the side of a smaller model. With more GPU time, I would experiment with larger hidden layers or more layers. I'd be cautious overfitting, but it would be worth exploring.
    - `num_epochs` : 2; I wanted to do at least 3, but I'm running low on GPU time, and trying to save some for the final project. More is usually better, as long as the model isn't overfitting.
    - `save_every`: 1; At least every epoch. I don't want to lose training data. I also added in save_every_steps=500 to save a bit more often. This helped save work, and let me test the models while training was still active. 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 
I left the existing transform. It seems quite reasonable:
    1) Resize(256): Resnet50 takes an image of size 224x224. This transform resizes to a bit larger than that, because it needs a bit of buffer for the RandomCrop that comes next.
    2) RandomCrop(224): By cropping random regions, we effectively translate/shift the image. This makes the model invariant to small changes in position of features. The size (224) has to be smaller than the original (256), but not too much smaller, or we risk cropping out relevant features that are mentioned in the caption.
    3) RandomHorizontalFlip(): This is This effectively doubles our input data by mirroring it. We only do it horizontally, because most things look very similar left-to-right, but quite different top-to-top. For example, a dog will always have a head at the top and legs on the bottom, but may be facing left or right.
    4) ToTensor(): Our model is expecting Tensor inputs
    5) Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))]): You should always normalize inputs. It helps the model to training to be much faster and more stable. These particular input parameters are using the mean and std-dev for Imagenet, a large image dataset. We could just as well have generated them on our dataset, but they presumably would have ended up very similar.


### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 
The pretrained CNN is the only thing we freeze. The rest of the model is new and needs to be trained so we aren't just using randomly initialized parameters. Freezing the pre-trained encoder layers saves lots of time, by not calculating gradients for backprop. Also, according to the "Show and Tell" paper, "changing them had a negative impact" anyways. 


### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:**
The "Show and Tell" paper used SGD, and the "Show, Attend, and Tell" paper used RMSProp and Adam. I almost always start with the Adam optimizer, since it incorporates the best of momentum and adaptive learning rates. It can be a bit more finnicky to tune, but it can train a lot faster, both because it's adaptive, and because the momentum terms help it roll through saddle points/regions of low grad. I also added a small weight decay term, to reduce overfitting. 

I played around with learning rate a bit. 0.01 was very fast initially, but my loss stopped decreased after a handful of steps, so I reduced it to 0.0005 for increased stability. That worked well, though I probably could have found more optimal values in between (again, didn't play too much, because I'm trying to conserve GPU time).

In [30]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
batch_size = 64          # batch size
vocab_threshold = 500        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings TODO: Show and tell uses 512, I think
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 2             # number of training epochs
save_every_epochs = 1             # determines frequency of saving model weights
save_every_steps = 500             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
# This step takes too long, and I keep re-running when I tune other params
# Skip by default
force_load = False 
if(not data_loader or force_load):
    data_loader = get_loader(transform=transform_train,
                             mode='train',
                             batch_size=batch_size,
                             vocab_threshold=vocab_threshold,
                             vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 

# TODO #4: Define the optimizer.
optimizer = torch.optim.Adam(params, betas=(.90,.999), lr=0.0005, weight_decay=1e-5) # consider reducing lr to 0.001 or 0.002

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [31]:
print('Total number of tokens in vocabulary:', len(data_loader.dataset.vocab))

Total number of tokens in vocabulary: 8855


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [29]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

running_loss = 5 # a bit arbitrary, but I know the initial loss is near this

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
response = requests.request("GET", 
                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
                            headers={"Metadata-Flavor":"Google"})

print(f"Training start time: {time.ctime()}")
for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            requests.request("POST", 
                             "https://nebula.udacity.com/api/v1/remote/keep-alive", 
                             headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        running_loss = .01*loss.item()+.99*running_loss # running ave (100ish steps)
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f, Running Loss: %.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()), running_loss)
        
        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            
        
        if i_step % save_every_steps == 0:
            decoder_name = os.path.join('./models', 'decoder-%d-%d.pkl' % (epoch, i_step))
            encoder_name = os.path.join('./models', 'encoder-%d-%d.pkl' % (epoch, i_step))
            torch.save(decoder.state_dict(), decoder_name)
            torch.save(encoder.state_dict(), encoder_name)
            print(f"Saving models: \n{decoder_name}\n{encoder_name}")
                   
    # Save the weights.
    if epoch % save_every_epochs == 0:
        decoder_name = os.path.join('./models', 'decoder-%d.pkl' % epoch)
        encoder_name = os.path.join('./models', 'encoder-%d.pkl' % epoch)
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
        print(f"Saving models: \n{decoder_name}\n{encoder_name}")

# Close the training log file.
f.close()

Training start time: Mon Nov  8 19:46:43 2021
Epoch [1/3], Step [100/6471], Loss: 3.7990, Perplexity: 44.6583, Running Loss: 3.8851
Epoch [1/3], Step [200/6471], Loss: 3.5361, Perplexity: 34.3318, Running Loss: 3.5952
Epoch [1/3], Step [300/6471], Loss: 3.6155, Perplexity: 37.1689, Running Loss: 3.4512
Epoch [1/3], Step [400/6471], Loss: 3.4058, Perplexity: 30.1379, Running Loss: 3.2982
Epoch [1/3], Step [500/6471], Loss: 2.9809, Perplexity: 19.7053, Running Loss: 3.0534
Saving models: 
./models/decoder-1-500.pkl
./models/encoder-1-500.pkl
Epoch [1/3], Step [600/6471], Loss: 3.4259, Perplexity: 30.7510, Running Loss: 3.26928
Epoch [1/3], Step [700/6471], Loss: 3.0822, Perplexity: 21.8067, Running Loss: 2.9931
Epoch [1/3], Step [800/6471], Loss: 2.6584, Perplexity: 14.2732, Running Loss: 2.9047
Epoch [1/3], Step [900/6471], Loss: 2.8452, Perplexity: 17.2044, Running Loss: 2.9797
Epoch [1/3], Step [1000/6471], Loss: 3.8012, Perplexity: 44.7533, Running Loss: 2.9215
Saving models: 
./mode

KeyboardInterrupt: 

In [ ]:
# Manual Save 
manual_save = False
epoch = 2
step = 3899
if manual_save:
    if step is None:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
    else:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d-%d.pkl' % (epoch, step)))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d-%d.pkl' % (epoch, step)))

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.